In [8]:
from jesse import research, helpers

warmup_candles, trading_candles = research.get_candles(
    'Binance Perpetual Futures', 'BTC-USDT', '1m',
    helpers.date_to_timestamp('2024-12-01'),
    helpers.date_to_timestamp('2024-12-31'),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

In [7]:
# timestamp, open, high, low, close, volume
trading_candles[0]

array([1.7330112e+12, 9.6475200e+04, 9.6428500e+04, 9.6475200e+04,
       9.6428500e+04, 4.3616000e+01])